# Spark RDD
https://catalog.us-east-1.prod.workshops.aws/workshops/ee59d21b-4cb8-4b3d-a629-24537cf37bb5/en-US/lab2/rdd

In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
3,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## # 外部データセットを使用して RDD を作成する


In [2]:
rdd = spark.sparkContext.textFile('s3://bucket001/input/lab2/sample.csv')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
type(rdd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.rdd.RDD'>

In [4]:
rdd.first()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'uuid,country,itemtype,saleschannel,orderpriority,orderdate,region,shipdate,unitssold,unitprice,unitcost,totalrevenue,totalcost,totalprofit'

In [5]:
rddheader = rdd.first()

rddwithoutheader = rdd.filter(lambda c: c != rddheader)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
type(rddwithoutheader.first())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'str'>

## ステップ 1 - 文字列をリストに変換する

In [7]:
# reading first record form rdd after removing header
s = rddwithoutheader.first()

# checkig type
type(s)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'str'>

In [8]:
# printing value of s
print(s)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

535113847,Azerbaijan,Snacks,Online,C,10/8/14,Middle East and North Africa,10/23/14,934,152.58,97.44,142509.72,91008.96,51500.76

In [9]:
# converting string into list
sl = s.split(',')

# checking type
type(sl)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'list'>

In [10]:
# printing list
for i in sl[0:len(sl)]:
    print(i)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

535113847
Azerbaijan
Snacks
Online
C
10/8/14
Middle East and North Africa
10/23/14
934
152.58
97.44
142509.72
91008.96
51500.76

## ステップ 2 - 異なる国を見つける

In [11]:
sl[1]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Azerbaijan'

In [12]:
rddcountry = rddwithoutheader.map(lambda r : r.split(',')[1]).distinct()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
rddcountry.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

185

In [14]:
for i in rddcountry.take(10):
    print(i)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Azerbaijan
Panama
Sao Tome and Principe
Belize
Denmark
Germany
Turkey
United Kingdom
Kazakhstan
Haiti

## ステップ 3 - 国が英国であるすべてのレコードをフィルター処理する

In [15]:
rddcountryuk = rddwithoutheader.filter(lambda o : o.split(',')[1] in ('United Kingdom'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
rddcountryuk.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

517

In [17]:
for i in rddcountryuk.take(10):
    print(i)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

847788178,United Kingdom,Snacks,Online,H,12/29/12,Europe,1/15/13,8217,152.58,97.44,1253749.86,800664.48,453085.38
757665097,United Kingdom,Clothes,Offline,C,5/7/15,Europe,5/14/15,8794,109.28,35.84,961008.32,315176.96,645831.36
579393041,United Kingdom,Personal Care,Offline,C,11/22/15,Europe,12/26/15,4131,81.73,56.67,337626.63,234103.77,103522.86
228189742,United Kingdom,Office Supplies,Offline,C,2/28/10,Europe,3/17/10,5270,651.21,524.96,3431876.7,2766539.2,665337.5
981146752,United Kingdom,Personal Care,Offline,H,6/14/14,Europe,7/20/14,1982,81.73,56.67,161988.86,112319.94,49668.92
361647021,United Kingdom,Personal Care,Online,L,3/1/12,Europe,3/4/12,8245,81.73,56.67,673863.85,467244.15,206619.7
136305415,United Kingdom,Personal Care,Online,L,10/13/12,Europe,11/24/12,4656,81.73,56.67,380534.88,263855.52,116679.36
675350463,United Kingdom,Baby Food,Offline,C,1/15/17,Europe,1/23/17,65,255.28,159.42,16593.2,10362.3,6230.9
981978023,United Kingdom,Household,Offline,C,4/3/10,Europe,4/13/10,79

## キーと値のペアの操作

In [18]:
## convert RDD into KeyValue table; here we have indexed "Item Type" and assign 1 to each "Item Type"; 
## this will help us to count each "Item Type"

rddcountryukkv = rddcountryuk.map(lambda r : (r.split(',')[2],1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# list 10 item_type 
rddcountryukkv.take(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('Snacks', 1), ('Clothes', 1), ('Personal Care', 1), ('Office Supplies', 1), ('Personal Care', 1), ('Personal Care', 1), ('Personal Care', 1), ('Baby Food', 1), ('Household', 1), ('Cereal', 1)]

In [20]:
## count the total "Item Type" by using reduceByKey

rddcountryukkvrkey = rddcountryukkv.reduceByKey(lambda a, b: a + b)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
## print item_type with count

for i in rddcountryukkvrkey.take(20):
    print(i)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Snacks', 44)
('Clothes', 46)
('Personal Care', 46)
('Office Supplies', 41)
('Baby Food', 39)
('Household', 46)
('Cereal', 39)
('Beverages', 48)
('Cosmetics', 45)
('Meat', 41)
('Vegetables', 42)
('Fruits', 40)

## Spark RDD の値のソート

In [22]:
## sorting the Item_Type with count; In Spark RDD you can use sortByKey to perform the sort

rddcountryukkvrkeysort = rddcountryukkvrkey.sortByKey()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
for i in rddcountryukkvrkeysort.take(10):
    print(i)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Baby Food', 39)
('Beverages', 48)
('Cereal', 39)
('Clothes', 46)
('Cosmetics', 45)
('Fruits', 40)
('Household', 46)
('Meat', 41)
('Office Supplies', 41)
('Personal Care', 46)

In [24]:
# if you see the previous output; it performed the sorting but on Item_Type 
# not on count; it is because sortByKey perform the sorting on Key instead of
# Value i.e. why now we need to perform one transform change the key value pair

rddcountryukkvrkey = rddcountryukkvrkey.map(lambda o : (o[1],o[0]))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
rddcountryukkvrkey.take(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(44, 'Snacks'), (46, 'Clothes'), (46, 'Personal Care'), (41, 'Office Supplies'), (39, 'Baby Food'), (46, 'Household'), (39, 'Cereal'), (48, 'Beverages'), (45, 'Cosmetics'), (41, 'Meat')]

In [26]:
## Now let's perform the sorting 
rddcountryukkvrkeysort = rddcountryukkvrkey.sortByKey(False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
for i in rddcountryukkvrkeysort.take(10):
    print(i)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(48, 'Beverages')
(46, 'Clothes')
(46, 'Personal Care')
(46, 'Household')
(45, 'Cosmetics')
(44, 'Snacks')
(42, 'Vegetables')
(41, 'Office Supplies')
(41, 'Meat')
(40, 'Fruits')

## データを S3 に保存する

In [28]:
## save the value to s3 as text/CSV file with Gzip compression

sc._jsc.hadoopConfiguration().set("mapred.output.committer.class", "org.apache.hadoop.mapred.DirectFileOutputCommitter")
codec = "org.apache.hadoop.io.compress.GzipCodec"
rddcountryukkvrkeysort.saveAsTextFile("s3://bucket001/input/lab2/result" + "/part-0000*" , codec)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## 項目タイプごとの総収益の合計を計算します

In [29]:
# In this part, we are going to calculate sum of total revenue per item type

# let's convert the data into key value pair to extract Item Type and Total Revenue  

rddcountryukkv = rddcountryuk.map(lambda sl : (sl.split(',')[2],float(sl.split(',')[11])))

# perform the sum by using reduceByKey()

rddcountryukkvggregateByKey = rddcountryukkv.reduceByKey(lambda total, value : total + value )

# print 10 values

for i in rddcountryukkvggregateByKey.take(10):
    print(i)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

('Snacks', 33305620.14)
('Clothes', 26638202.080000002)
('Personal Care', 19192247.249999996)
('Office Supplies', 133824306.21)
('Baby Food', 46536522.88000001)
('Household', 165361406.69000003)
('Cereal', 42633382.0)
('Beverages', 11407217.249999998)
('Cosmetics', 109017131.60000002)
('Meat', 80643007.83000001)

## RDD からの DataFrame の変換

In [30]:
# run one by one
type(rddcountryukkvggregateByKey)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.rdd.PipelinedRDD'>

In [31]:
rddcountryukkvggregateByKeyDF = rddcountryukkvggregateByKey.toDF()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
type(rddcountryukkvggregateByKeyDF)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [33]:
rddcountryukkvggregateByKeyDF.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _1: string (nullable = true)
 |-- _2: double (nullable = true)

In [34]:
# Create a schema for the dataframe

from pyspark.sql.types import StructField, StructType, StringType, IntegerType , FloatType

schema = StructType([
    StructField('Item Type', StringType(), True),
    StructField('Sum of Total Revenue', FloatType(), True)
])

# Create data frame

df = spark.createDataFrame(rddcountryukkvggregateByKey,schema)

# show data frame

df.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------+--------------------+
|      Item Type|Sum of Total Revenue|
+---------------+--------------------+
|         Snacks|          3.330562E7|
|        Clothes|         2.6638202E7|
|  Personal Care|         1.9192248E7|
|Office Supplies|        1.33824304E8|
|      Baby Food|         4.6536524E7|
|      Household|        1.65361408E8|
|         Cereal|         4.2633384E7|
|      Beverages|         1.1407217E7|
|      Cosmetics|        1.09017128E8|
|           Meat|         8.0643008E7|
|     Vegetables|         3.2463986E7|
|         Fruits|           1491064.6|
+---------------+--------------------+